Link repositorio de GitHub: ## Repositorio en GitHub

Puedes encontrar todos los archivos en el siguiente enlace:

[Repositorio FotoApp en GitHub](https://github.com/Shinkuu0/fotoapp-proyecto/tree/main/fotoapp-proyecto)

In [1]:
from PIL import Image, ImageEnhance, ImageFilter
import urllib.request
from io import BytesIO
import numpy as np
import os

def descargar_imagen(url):
    try:
        with urllib.request.urlopen(url) as url_response:
            imagen = Image.open(BytesIO(url_response.read()))
        return imagen
    except Exception as e:
        print(f"Error al descargar la imagen: {e}")
        raise

def redimensionar_para_red(imagen, red_social):
    try:
        dimensiones = {
            "youtube": (1280, 720),
            "instagram": (1080, 1080),
            "twitter": (1200, 675),
            "facebook": (1200, 630)
        }

        red_social = red_social.lower()
        if red_social not in dimensiones:
            raise ValueError(f"Red social no soportada. Opciones válidas: {list(dimensiones.keys())}")

        ancho_objetivo, alto_objetivo = dimensiones[red_social]

        ratio_original = imagen.width / imagen.height
        ratio_objetivo = ancho_objetivo / alto_objetivo

        if ratio_original > ratio_objetivo:
            nuevo_ancho = ancho_objetivo
            nuevo_alto = int(nuevo_ancho / ratio_original)
        else:
            nuevo_alto = alto_objetivo
            nuevo_ancho = int(nuevo_alto * ratio_original)

        imagen_redimensionada = imagen.resize((nuevo_ancho, nuevo_alto), Image.Resampling.LANCZOS)
        imagen_final = Image.new("RGB", dimensiones[red_social], (255, 255, 255))
        offset = ((ancho_objetivo - nuevo_ancho) // 2, (alto_objetivo - nuevo_alto) // 2)
        imagen_final.paste(imagen_redimensionada, offset)

        return imagen_final
    except Exception as e:
        print(f"Error al procesar la imagen: {str(e)}")
        raise

def ajustar_contraste(imagen):
    try:
        imagen_ycbcr = imagen.convert('YCbCr')
        canales = list(imagen_ycbcr.split())

        canal_y = np.array(canales[0])
        hist, bins = np.histogram(canal_y.flatten(), 256, [0, 256])
        cdf = hist.cumsum()
        cdf_normalizado = cdf * float(hist.max()) / cdf.max()

        lookup_table = np.interp(canal_y.flatten(), bins[:-1], cdf_normalizado)
        canales[0] = Image.fromarray(lookup_table.reshape(canal_y.shape).astype('uint8'))

        imagen_final = Image.merge('YCbCr', canales).convert('RGB')
        return imagen_final
    except Exception as e:
        print(f"Error al ajustar el contraste: {str(e)}")
        raise

def aplicar_filtro(imagen, opcion_filtro):
    """Aplica un filtro seleccionado por número (incluyendo la opción 'original')."""
    try:
        filtros = {
            1: 'original',
            2: ImageFilter.BLUR,
            3: ImageFilter.CONTOUR,
            4: ImageFilter.DETAIL,
            5: ImageFilter.EDGE_ENHANCE,
            6: ImageFilter.EDGE_ENHANCE_MORE,
            7: ImageFilter.EMBOSS,
            8: ImageFilter.FIND_EDGES,
            9: ImageFilter.SHARPEN,
            10: ImageFilter.SMOOTH
        }

        if opcion_filtro == 1:
            return imagen

        if opcion_filtro in filtros:
            imagen_filtrada = imagen.filter(filtros[opcion_filtro])
            return imagen_filtrada
        else:
            raise ValueError("Opción de filtro no válida")

    except Exception as e:
        print(f"Error al aplicar el filtro: {str(e)}")
        raise

def boceto_persona(imagen, persona=True):
    try:
        if not persona:
            raise ValueError("La imagen no contiene una persona")

        imagen_gris = imagen.convert('L')
        bordes = imagen_gris.filter(ImageFilter.FIND_EDGES)
        enhancer = ImageEnhance.Contrast(bordes)
        boceto = enhancer.enhance(2.0)
        boceto = ImageEnhance.Brightness(boceto).enhance(2.0)

        return boceto
    except Exception as e:
        print(f"Error al crear el boceto: {str(e)}")
        raise

def menu_principal():
    while True:
        print("\n=== FotoApp Menu ===")
        print("1. Redimensionar para red social")
        print("2. Ajustar contraste")
        print("3. Aplicar filtro")
        print("4. Crear boceto artístico")
        print("5. Salir")

        try:
            opcion = input("\nElige una opción (1-5): ")

            if opcion == "5":
                print("¡Gracias por usar FotoApp!")
                break

            url_imagen = input("Ingresa la URL de la imagen: ")
            imagen = descargar_imagen(url_imagen)

            if opcion == "1":
                red_social = input("Ingresa la red social (Youtube/Instagram/Twitter/Facebook): ")
                imagen = redimensionar_para_red(imagen, red_social)
                imagen.show()

            elif opcion == "2":
                imagen = ajustar_contraste(imagen)
                imagen.show()

            elif opcion == "3":
                print("\n=== FotoApp: Aplicación de Filtros ===")
                print("Elige una opción de filtro (1-10):")
                print("1. Original")
                print("2. Blur")
                print("3. Contour")
                print("4. Detail")
                print("5. Edge Enhance")
                print("6. Edge Enhance More")
                print("7. Emboss")
                print("8. Find Edges")
                print("9. Sharpen")
                print("10. Smooth")

                opcion_filtro = int(input("\nSelecciona el número del filtro: "))

                imagen_filtrada = aplicar_filtro(imagen, opcion_filtro)
                imagen_filtrada.show()

                nombre_filtro = ['original', 'blur', 'contour', 'detail', 'edge_enhance', 'edge_enhance_more', 'emboss', 'find_edges', 'sharpen', 'smooth']
                nombre_salida = f"{nombre_filtro[opcion_filtro - 1]}_{os.path.basename(url_imagen)}"
                imagen_filtrada.save(nombre_salida)

                print(f"Imagen guardada como: {nombre_salida}")

            elif opcion == "4":
                respuesta = input("¿La imagen contiene una persona? (s/n): ").lower()
                persona = respuesta == 's'
                imagen = boceto_persona(imagen, persona)
                imagen.show()

            else:
                print("Opción no válida. Por favor elige una opción entre 1 y 5.")

        except ValueError as e:
            print(f"Error: {e}")
        except Exception as e:
            print(f"Ocurrió un error inesperado: {e}")

menu_principal()



=== FotoApp: Aplicación de Filtros ===
Elige una opción de filtro (1-10):
1. Original
2. Blur
3. Contour
4. Detail
5. Edge Enhance
6. Edge Enhance More
7. Emboss
8. Find Edges
9. Sharpen
10. Smooth
Error: unknown file extension: 

=== FotoApp Menu ===
1. Redimensionar para red social
2. Ajustar contraste
3. Aplicar filtro
4. Crear boceto artístico
5. Salir
¡Gracias por usar FotoApp!
